<a href="https://colab.research.google.com/github/madham97/Least_Miserables/blob/main/Hammad/RAG_initial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# !pip install biopython
# !pip install transformers torch
# !pip install sentence-transformers
# !pip install openai
# !pip install python-dotenv

In [19]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from Bio import Entrez
import os
from dotenv import set_key, dotenv_values
from sentence_transformers import SentenceTransformer, util

In [20]:
Entrez.email = "hammad.aamer97@gmail.com"
Entrez.api_key = "b3d2a3e731843a8ac6a4664c0c103ef7c308"

query = "intelligence"
start_date = datetime(2013, 1, 1)
end_date = datetime(2016, 1, 3)

print(f"Start date: {start_date}")
print(f"End date: {end_date}")

batch_size = 100
abstracts = []

# Iterate over months
current_date = start_date
while current_date < end_date:
    next_date = current_date + timedelta(days=30)

    current_date_str = current_date.strftime("%Y/%m/%d")
    next_date_str = next_date.strftime("%Y/%m/%d")

    # Perform the search to get the PubMed IDs
    search_query = f"({query}) AND ({current_date_str}[Date - Publication] : {next_date_str}[Date - Publication])"
    handle = Entrez.esearch(db="pubmed", term=search_query)
    record = Entrez.read(handle)
    handle.close()

    pubmed_id_count = record['Count']

    # Retrieve the abstracts for the retrieved PubMed IDs
    handle = Entrez.esearch(db="pubmed", term=search_query, retmax=pubmed_id_count)
    record = Entrez.read(handle)
    handle.close()

    pubmed_ids = record['IdList']
    print(f"Total articles for {current_date_str} to {next_date_str}: {len(pubmed_ids)}\n")

    # Retrieve abstracts in batches
    for retstart in range(0, int(pubmed_id_count), batch_size):
        batch_pubmed_ids = record['IdList'][retstart:retstart + batch_size]
        print(f"Batch: {(retstart/batch_size)+1}/{(len(pubmed_ids)/batch_size)//1+1}")

        handle = Entrez.efetch(db="pubmed", id=batch_pubmed_ids, rettype="abstract", retmode="text")
        batch_abstracts = handle.read()
        abstracts.append(batch_abstracts)
        handle.close()

    # Move to the next month
    current_date = next_date

# Now 'abstracts' contains the abstracts retrieved over the specified date range in monthly batches


Start date: 2013-01-01 00:00:00
End date: 2020-01-03 00:00:00
Total articles for 2013/01/01 to 2013/01/31: 2342

Batch: 1.0/24.0
Batch: 2.0/24.0
Batch: 3.0/24.0
Batch: 4.0/24.0
Batch: 5.0/24.0
Batch: 6.0/24.0
Batch: 7.0/24.0
Batch: 8.0/24.0
Batch: 9.0/24.0
Batch: 10.0/24.0
Batch: 11.0/24.0
Batch: 12.0/24.0
Batch: 13.0/24.0
Batch: 14.0/24.0
Batch: 15.0/24.0
Batch: 16.0/24.0
Batch: 17.0/24.0
Batch: 18.0/24.0
Batch: 19.0/24.0
Batch: 20.0/24.0
Batch: 21.0/24.0
Batch: 22.0/24.0
Batch: 23.0/24.0
Batch: 24.0/24.0
Total articles for 2013/01/31 to 2013/03/02: 1275

Batch: 1.0/13.0
Batch: 2.0/13.0
Batch: 3.0/13.0
Batch: 4.0/13.0
Batch: 5.0/13.0
Batch: 6.0/13.0
Batch: 7.0/13.0
Batch: 8.0/13.0
Batch: 9.0/13.0
Batch: 10.0/13.0
Batch: 11.0/13.0
Batch: 12.0/13.0
Batch: 13.0/13.0
Total articles for 2013/03/02 to 2013/04/01: 818

Batch: 1.0/9.0
Batch: 2.0/9.0
Batch: 3.0/9.0
Batch: 4.0/9.0
Batch: 5.0/9.0
Batch: 6.0/9.0
Batch: 7.0/9.0
Batch: 8.0/9.0
Batch: 9.0/9.0
Total articles for 2013/04/01 to 2013/05

In [21]:
import re

# Function to split a batch into individual abstracts
def split_batch(batch):
    # Use regular expression to split the batch into an array of abstracts
    abstracts = re.split(r'\n\n\d+\. ', batch.strip())

    # Remove empty strings from the list
    abstracts = list(filter(None, abstracts))

    return abstracts

# Split each entry in the 'abstracts' array into an array of individual abstracts
individual_abstracts = [split_batch(batch) for batch in abstracts]

# Flatten the list of lists into a single list
individual_abstracts = [abstract for batch in individual_abstracts for abstract in batch]

# # Print the result
# for i, abstract in enumerate(individual_abstracts, start=1):
#     print(f"Abstract {i}:\n{abstract.strip()}\n")


In [22]:
print(len(individual_abstracts))

117761


In [23]:
df = pd.DataFrame({'Abstracts': abstracts})

# Save the DataFrame to an Excel file
excel_filename = 'pubmed_abstracts.xlsx'
df.to_excel(excel_filename, index=False)

In [ ]:
from sentence_transformers import SentenceTransformer, util
from transformers import BartForConditionalGeneration, BartTokenizer

class DenseVectorRetrieval:
    def __init__(self, documents):
        self.documents = documents
        self.model = SentenceTransformer("paraphrase-mpnet-base-v2")

        # Encode documents into dense vectors
        self.document_embeddings = self.model.encode(documents)

    def retrieve_information(self, query, top_k=3):
        # Encode the query into a dense vector
        query_embedding = self.model.encode(query)

        # Calculate cosine similarity between the query and document embeddings
        similarities = util.pytorch_cos_sim(query_embedding, self.document_embeddings)[0]

        # Get indices of top-k most similar documents
        top_k_indices = similarities.argsort(descending=True)[:top_k]

        # Retrieve the top-k documents
        matching_documents = [self.documents[i] for i in top_k_indices]

        return matching_documents

class RetrievalAugmentedGenerator:
    def __init__(self, retrieval_component, generator_model, tokenizer):
        self.retrieval_component = retrieval_component
        self.generator_model = generator_model
        self.tokenizer = tokenizer

    def generate_text_with_retrieval(self, query):
        # Retrieve relevant information using the retrieval component
        retrieved_information = self.retrieval_component.retrieve_information(query)

        # Concatenate the query and retrieved information for generation
        input_text = f"{query} Relevant information: {' '.join(retrieved_information)}"

        # Tokenize and generate text using the language model
        input_ids = self.tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
        generated_output = self.generator_model.generate(input_ids, max_length=150, num_return_sequences=1, no_repeat_ngram_size=2, top_k=50, top_p=0.95)

        # Decode and return the generated text
        generated_text = self.tokenizer.decode(generated_output[0], skip_special_tokens=True)
        return generated_text

# Example usage
documents = individual_abstracts

retrieval_component = DenseVectorRetrieval(documents)


In [ ]:
# Load pre-trained BART model and tokenizer
generator_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

# Create RetrievalAugmentedGenerator
rag_model = RetrievalAugmentedGenerator(retrieval_component, generator_model, tokenizer)

# Sample query
query = "What diseases are the most dangerous"

# Generate text with retrieval augmentation
generated_text = rag_model.generate_text_with_retrieval(query)

# Display the generated text
print("Generated Text:")
print(generated_text)
